# Performance Estimates for the Israeli Inventors Dataset

This notebook showcases the use of our precision and recall performance estimators in application to the Israeli inventors benchmark dataset.

Note that the Israeli dataset only covers patents granted between 1963 and 1999. As such, we can only estimate the performance of the current disambiguation algorithm for this time period.

Furthermore, we consider the the Israeli benchmark to be following the "single block" sampling process.

In [1]:
from pv_evaluation.estimators import pairwise_precision_estimator, pairwise_recall_estimator, pairwise_precision_std
from pv_evaluation.benchmark import load_israeli_inventors_benchmark

In [2]:
import pandas as pd
import numpy as np
import wget
import zipfile
import os

if not os.path.isfile("rawinventor.tsv"):
    wget.download("https://s3.amazonaws.com/data.patentsview.org/download/rawinventor.tsv.zip")
    with zipfile.ZipFile("rawinventor.tsv.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    os.remove("rawinventor.tsv.zip")

if not os.path.isfile("patent.tsv"):
    wget.download("https://s3.amazonaws.com/data.patentsview.org/download/patent.tsv.zip")
    with zipfile.ZipFile("patent.tsv.zip", 'r') as zip_ref:
        zip_ref.extractall(".")
    os.remove("patent.tsv.zip")

In [3]:
patent = pd.read_csv("patent.tsv", sep="\t", dtype=str, usecols=["id", "date"])
rawinventor = pd.read_csv("rawinventor.tsv", sep="\t", dtype=str, usecols=["patent_id", "sequence", "inventor_id"])

date = pd.DatetimeIndex(patent.date)
patent["date"] = date.year.astype(int)
joined = rawinventor.merge(patent, left_on="patent_id", right_on="id", how="left")

In [4]:
joined["mention-id"] = "US" + joined.patent_id + "-" + joined.sequence
joined = joined.query('date >= 1963 and date <= 1999')
current_disambiguation = joined.set_index("mention-id")["inventor_id"]

In [5]:
pairwise_precision_estimator(current_disambiguation, load_israeli_inventors_benchmark(), sampling_type="single_block", weights="uniform")

0.7860646822490067

In [6]:
pairwise_recall_estimator(current_disambiguation, load_israeli_inventors_benchmark(), sampling_type="single_block", weights="uniform")

0.9409842164000638